In [40]:
from bootcamp.databricks_compat import display, dbutils, spark
from data.config import DATA_DIR

In [42]:
diamonds_df = spark.read.format("csv").option("header", "true").load(f"{DATA_DIR}/databricks-datasets/Rdatasets/csv/ggplot2/diamonds.csv")

In [43]:
diamonds_df.write.format("delta").mode("overwrite").saveAsTable("diamond")


In [44]:
diamonds_df.count()

53940

In [45]:
diamonds_df.show()

+--------+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|rownames|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
+--------+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|       1| 0.23|    Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|
|       2| 0.21|  Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|
|       3| 0.23|     Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|
|       4| 0.29|  Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|
|       5| 0.31|     Good|    J|    SI2| 63.3|   58|  335|4.34|4.35|2.75|
|       6| 0.24|Very Good|    J|   VVS2| 62.8|   57|  336|3.94|3.96|2.48|
|       7| 0.24|Very Good|    I|   VVS1| 62.3|   57|  336|3.95|3.98|2.47|
|       8| 0.26|Very Good|    H|    SI1| 61.9|   55|  337|4.07|4.11|2.53|
|       9| 0.22|     Fair|    E|    VS2| 65.1|   61|  337|3.87|3.78|2.49|
|      10| 0.23|Very Good|    H|    VS1| 59.4|   61|  338|   4|4.05|2.39|
|      11|  0.3|     Good|    J|    SI

In [46]:
spark.sql("select count(*) from diamond").show()

+--------+
|count(1)|
+--------+
|   53940|
+--------+



In [47]:
%%sql
select count(*) from diamond


count(1)
53940


In [48]:
%%sql

DESCRIBE DETAIL diamond

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,1400990b-8e48-4b87-8448-7c34cf1be580,spark_catalog.default.diamond,None,file:/home/jovyan/work/data/spark-warehouse/diamon...,2025-12-09 12:53:47.940000,2025-12-09 12:53:48.397000,[],[],1,755317,{},1,2,"['appendOnly', 'invariants']"


In [51]:
%fs ls /user/hive/warehouse/diamond

FileInfo(path='dbfs:/home/jovyan/work/data/spark-warehouse/diamond/.part-00000-15c4194a-a3ff-4218-9a3c-6146be5c597d-c000.snappy.parquet.crc', name='.part-00000-15c4194a-a3ff-4218-9a3c-6146be5c597d-c000.snappy.parquet.crc', size=5912)
FileInfo(path='dbfs:/home/jovyan/work/data/spark-warehouse/diamond/_delta_log', name='_delta_log/', size=0)
FileInfo(path='dbfs:/home/jovyan/work/data/spark-warehouse/diamond/part-00000-15c4194a-a3ff-4218-9a3c-6146be5c597d-c000.snappy.parquet', name='part-00000-15c4194a-a3ff-4218-9a3c-6146be5c597d-c000.snappy.parquet', size=755317)


In [52]:
%%sql

DESCRIBE TABLE diamond

col_name,data_type,comment
rownames,string,None
carat,string,None
cut,string,None
color,string,None
clarity,string,None
depth,string,None
table,string,None
price,string,None
x,string,None
y,string,None


In [53]:
from pyspark.sql.functions import avg

filtered_df = diamonds_df.filter((diamonds_df["cut"] == "Ideal") & (diamonds_df["price"] > 1000))
grouped_df = filtered_df.groupBy("clarity").agg(avg("price"))
ordered_df = grouped_df.orderBy(grouped_df["avg(price)"].desc())

In [54]:
ordered_df.show()

+-------+------------------+
|clarity|        avg(price)|
+-------+------------------+
|    SI2| 5457.046354635463|
|    SI1| 4890.193991690636|
|    VS1| 4820.202886597938|
|    VS2| 4744.693523396343|
|     I1| 4642.622222222222|
|   VVS2| 4598.633727810651|
|   VVS1|   3575.6672091131|
|     IF|3532.8973561430794|
+-------+------------------+



In [55]:
%%sql
SELECT clarity, AVG(price) as avg_price
FROM (
  SELECT *
  FROM diamond
  WHERE cut = 'Ideal' AND price > 1000
)
GROUP BY clarity
ORDER BY avg_price DESC

clarity,avg_price
SI2,5457.046354635463
SI1,4890.193991690636
VS1,4820.202886597938
VS2,4744.693523396343
I1,4642.622222222222
VVS2,4598.633727810651
VVS1,3575.6672091131
IF,3532.8973561430794
